# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
from config import api_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
file= '../Weather_Jake.py/weather_data.csv'
weather_data_df= pd.read_csv(file)

weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chapais,49.7834,-74.8492,46.40,31,75,14.97,CA,1619901681
1,bluff,-46.6000,168.3333,48.99,83,57,3.00,NZ,1619901682
2,at-bashi,41.1702,75.8106,51.89,65,100,2.57,KG,1619901682
3,kiunga,-6.1219,141.2906,72.14,99,92,1.63,PG,1619901683
4,mataura,-46.1927,168.8643,48.99,83,17,3.00,NZ,1619901684
...,...,...,...,...,...,...,...,...,...
565,amazar,53.8580,120.8750,38.86,78,96,0.96,RU,1619902057
566,qiqihar,47.3408,123.9672,46.11,62,11,5.32,CN,1619902057
567,vila do maio,15.1333,-23.2167,71.60,73,20,23.02,CV,1619902058
568,ilulissat,69.2167,-51.1000,46.40,42,20,12.66,GL,1619902059


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#capture lat and long
locations= weather_data_df[['Lat', 'Lng']]

humidity_weight= weather_data_df['Humidity']


In [28]:
#plot heatmap with gmaps
fig = gmaps.figure()

#create heat layer on top of gmaps
#make weight of heatmap reflect humidity
heat_layer= gmaps.heatmap_layer(locations, weights=humidity_weight, dissipating=False, max_intensity=100,
                                 point_radius=1.5)

#add layer to figure
fig.add_layer(heat_layer)

#Display combined
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
#reference main df
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chapais,49.7834,-74.8492,46.40,31,75,14.97,CA,1619901681
1,bluff,-46.6000,168.3333,48.99,83,57,3.00,NZ,1619901682
2,at-bashi,41.1702,75.8106,51.89,65,100,2.57,KG,1619901682
3,kiunga,-6.1219,141.2906,72.14,99,92,1.63,PG,1619901683
4,mataura,-46.1927,168.8643,48.99,83,17,3.00,NZ,1619901684


In [30]:
hotel_df =weather_data_df.loc[(weather_data_df['Max Temp'] > 65) &
                    (weather_data_df['Max Temp'] < 87) &
                    (weather_data_df['Wind Speed'] < 7) &
                    (weather_data_df['Cloudiness'] < 10)]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,atuona,-9.8000,-139.0333,79.02,70,2,6.31,PF,1619901697
31,saint-philippe,-21.3585,55.7679,75.20,69,0,6.91,RE,1619901699
90,kavali,14.9167,79.9833,79.68,81,0,5.99,IN,1619901747
136,nanning,22.8167,108.3167,75.20,88,0,4.47,CN,1619901781
137,abu dhabi,24.4667,54.3667,80.60,83,0,5.75,AE,1619901714
139,gat,31.6100,34.7642,71.01,31,1,2.13,IL,1619901781
163,havelock,34.8791,-76.9013,71.01,34,0,1.01,US,1619901789
187,corrales,35.2378,-106.6067,81.00,16,1,4.61,US,1619901808
194,teknaf,20.8624,92.3058,77.13,85,0,1.52,BD,1619901813
221,urubicha,-15.3833,-62.9500,81.39,60,6,2.10,BO,1619901838


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
#reference
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,atuona,-9.8000,-139.0333,79.02,70,2,6.31,PF,1619901697
31,saint-philippe,-21.3585,55.7679,75.20,69,0,6.91,RE,1619901699
90,kavali,14.9167,79.9833,79.68,81,0,5.99,IN,1619901747
136,nanning,22.8167,108.3167,75.20,88,0,4.47,CN,1619901781
137,abu dhabi,24.4667,54.3667,80.60,83,0,5.75,AE,1619901714


In [32]:
hotel_df['Hotel Name']= ''
hotel_df.head()

C:\Users\burns\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,atuona,-9.8000,-139.0333,79.02,70,2,6.31,PF,1619901697,
31,saint-philippe,-21.3585,55.7679,75.20,69,0,6.91,RE,1619901699,
90,kavali,14.9167,79.9833,79.68,81,0,5.99,IN,1619901747,
136,nanning,22.8167,108.3167,75.20,88,0,4.47,CN,1619901781,
137,abu dhabi,24.4667,54.3667,80.60,83,0,5.75,AE,1619901714,


In [33]:
#call using Google Places API

#https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
#output is json  so:

#base url to be used later:   https://maps.googleapis.com/maps/api/place/nearbysearch/json (parameters)

#API DOC SITE: https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests

In [34]:
# set base_url to be used with parameters

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [35]:
#api call requires parameters to be set up

# params dictionary to update each iteration
paramaters = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

###### Test API Call URL:

In [36]:
#test url

#don't print with api key on public repository

#         get lat, lng from df
# lat_test = -21.3585
# lng_test = 55.7679


# paramaters['location']= f"{lat_test},{lng_test}"

#           make request and print url
# name_address = requests.get(base_url, params=paramaters)
    
#          print the name_address url, avoid doing for public github repos in order to avoid exposing key
#print(name_address.url)


In [37]:
#create test df of nly 5 cities to test when making first call to ensure code works


#this allows you to check code without making too many pulls

test_df = hotel_df.head(1)
test_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,atuona,-9.8,-139.0333,79.02,70,2,6.31,PF,1619901697,


#### Once code is confirmed to work, replace test_df  with  hotel_df and collect hotel name for all cities:

In [38]:
# use for loop with api call to obtain all cities in df using hotel_df

# .index neaded to grab value at df location

for near_hotel in hotel_df.index:
    # get lat, lng from df
    #iterate through each row(near_hotel) 
    lat = hotel_df["Lat"][near_hotel]
    lng = hotel_df["Lng"][near_hotel]
    
    # change location each iteration while leaving original params in place
    #adds to existing parameter dictionary
    paramaters["location"] = f"{lat},{lng}"
    
    #request url and convert to json
    #request_hotel= (requests.get(base_url, params=paramaters))
    request_json_hotel= (requests.get(base_url, params=paramaters)).json()
    

    #look at json for rfeference
#print(json.dumps(request_json_hotel, indent=4))
    
    
    
    #use try and except to keep code running even if data is missing
    
    #include near_hotel as the index for .loc
    try:
        hotel_df.loc[near_hotel, 'Hotel Name'] = request_json_hotel['results'][0]['name']
        
    except (KeyError, IndexError):
        print('Data Error.....skipping.')
    



C:\Users\burns\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Data Error.....skipping.
Data Error.....skipping.
Data Error.....skipping.


In [39]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,atuona,-9.8000,-139.0333,79.02,70,2,6.31,PF,1619901697,Villa Enata
31,saint-philippe,-21.3585,55.7679,75.20,69,0,6.91,RE,1619901699,"Chambres d'hôte ""La Trinité"""
90,kavali,14.9167,79.9833,79.68,81,0,5.99,IN,1619901747,Lic B.M Quearter's
136,nanning,22.8167,108.3167,75.20,88,0,4.47,CN,1619901781,Nanning Travelling With Hostel
137,abu dhabi,24.4667,54.3667,80.60,83,0,5.75,AE,1619901714,Ramee Garden Hotel Apartments
139,gat,31.6100,34.7642,71.01,31,1,2.13,IL,1619901781,OlusHome
163,havelock,34.8791,-76.9013,71.01,34,0,1.01,US,1619901789,Sherwood Motel
187,corrales,35.2378,-106.6067,81.00,16,1,4.61,US,1619901808,Quality Inn
194,teknaf,20.8624,92.3058,77.13,85,0,1.52,BD,1619901813,Forestry Guesthouse
221,urubicha,-15.3833,-62.9500,81.39,60,6,2.10,BO,1619901838,


In [40]:
clean_hotel_df = hotel_df.dropna(subset= ['Hotel Name'])
clean_hotel_df.replace('', np.nan, inplace=True)

#try and drop the NaN
clean_hotel_df.dropna()
clean_hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,atuona,-9.8000,-139.0333,79.02,70,2,6.31,PF,1619901697,Villa Enata
31,saint-philippe,-21.3585,55.7679,75.20,69,0,6.91,RE,1619901699,"Chambres d'hôte ""La Trinité"""
90,kavali,14.9167,79.9833,79.68,81,0,5.99,IN,1619901747,Lic B.M Quearter's
136,nanning,22.8167,108.3167,75.20,88,0,4.47,CN,1619901781,Nanning Travelling With Hostel
137,abu dhabi,24.4667,54.3667,80.60,83,0,5.75,AE,1619901714,Ramee Garden Hotel Apartments
139,gat,31.6100,34.7642,71.01,31,1,2.13,IL,1619901781,OlusHome
163,havelock,34.8791,-76.9013,71.01,34,0,1.01,US,1619901789,Sherwood Motel
187,corrales,35.2378,-106.6067,81.00,16,1,4.61,US,1619901808,Quality Inn
194,teknaf,20.8624,92.3058,77.13,85,0,1.52,BD,1619901813,Forestry Guesthouse
221,urubicha,-15.3833,-62.9500,81.39,60,6,2.10,BO,1619901838,NaN


In [41]:
clean_hotel_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
Hotel Name     object
dtype: object

In [42]:
clean_hotel_df.dropna(inplace=True)

In [43]:



# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = '''
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
'''
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
marker_locations = clean_hotel_df[["Lat", "Lng"]]






In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info, display_info_box=True)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))